IMPORTS

In [15]:
import main
import importlib
import scrape.harris.harris_county_scraper
from dotenv import load_dotenv
import ocr.ocr
import os
import string

RELOAD MODULES

In [27]:
load_dotenv()
importlib.reload(main)
importlib.reload(ocr.ocr)
importlib.reload(scrape.harris.harris_county_scraper)

<module 'scrape.harris.harris_county_scraper' from '/Users/isaaclam/guardian/marketing_leads_project/main/scrape/harris/harris_county_scraper.py'>

RUN MAIN

In [29]:
# run main
main.main()

Ensuring download directory is empty before processing the next case...
Processing case number: 1243926...
Document matching criteria not found; attempting to download the largest available document.
Adding case_number: 1243926 to no-match log.
[2025-02-05 14:47:59] Downloading largest document (size: 8684758 bytes) to /Users/isaaclam/guardian/marketing_leads_project/main/out/harris/downloaded_docs
Combined details: "Tyrell Brewer", "11500 KEEGANS RIDGE RD APT 902 HOUSTON TX 77031-2526", "Continental Casualty Company", "Osborn, Brett T. Abbott Osborn Jacobs PLC 974 - 73rd Street, Suite 20 West Des Moines IA 50265 Phone 800-617-7593"
Extracting text from PDF...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing

KeyboardInterrupt: 

TEST PDF PROCESSING

In [ ]:
ocr.ocr.process_pdf_and_find_damages('ocr/example_docs/sample1.pdf')

TEST CSV CONVERSION

In [4]:
input_txt = '/Users/isaaclam/guardian/marketing_leads_project/main/out/harris/defendant_data.txt'
output_csv = '/Users/isaaclam/guardian/marketing_leads_project/main/out/harris/defendant_data.csv'
main.convert_txt_to_csv(input_txt, output_csv)

TEST PRE-PROCESSING

In [5]:
pdf_path = '/Users/isaaclam/guardian/marketing_leads_project/main/ocr/example_docs/sample1.pdf'
output_folder = 'processed_images'
pages = ocr.ocr.convert_from_path(pdf_path, dpi=300)
for page_number,page_image in enumerate(pages,start=1):
    ocr.ocr.preprocess_image_to_remove_watermark(page_image, output_folder, page_number)

PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file '/Users/isaaclam/guardian/marketing_leads_project/main/ocr/example_docs/sample1.pdf': No such file or directory.


In [150]:
pdf_path = '/Users/isaaclam/guardian/marketing_leads_project/main/ocr/example_docs/sample1.pdf'

if not os.path.exists(pdf_path):
    raise FileNotFoundError("PDF file not found. Please check the path.")

print("Extracting text from PDF...")
extracted_text = ocr.ocr.extract_text_from_pdf_with_watermark_removal(pdf_path)

print("\nSearching for 'damages' and the associated dollar value...")
# result = find_damages_and_value(extracted_text)

Extracting text from PDF...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing pa

MANUALLY VERIFY CSV

In [80]:
verified_csv = '/Users/isaaclam/guardian/marketing_leads_project/main/out/harris/defendant_data_verified.csv'
output_csv = '/Users/isaaclam/guardian/marketing_leads_project/main/out/harris/defendant_data.csv'
filtered_csv = '/Users/isaaclam/guardian/marketing_leads_project/main/out/harris/defendant_data_verified_filtered.csv'
main.verify_csv(output_csv, verified_csv,filtered_csv)

TEST OCR AGAINST MANUALLY CREATED DATASET

In [18]:
def getDollarAmount(input):
    dollar_amount = None
    for word in input.split():
        if word.startswith('$'):
            # Strip trailing punctuation (but keep the initial '$')
            dollar_amount = word.rstrip(string.punctuation)
            break
    return dollar_amount if dollar_amount else "No dollar amount found."

In [6]:
mainDir = '/Users/isaaclam/guardian/marketing_leads_project/main'
outDir = f'{mainDir}/test/ocr/out'
pdfDir = f'{mainDir}/test/ocr/test_data_set'

In [ ]:
with open(f'{mainDir}/test/ocr/test_results_dataset.txt','r',encoding='utf-8') as f:
    lines = f.readlines()

for i in range (1,90):
    pdfStr =f'{pdfDir}/{i}.pdf'
    res = ocr.ocr.process_pdf_and_find_damages(pdfStr,False)
    
    dollarAmount = getDollarAmount(res)
    splitRes = res.split(',')
    courtName = splitRes[len(splitRes)-1]
    courtNameSplit = courtName.split(' ')
    courtNumber = courtNameSplit[len(courtNameSplit)-2]
    with open(f'{outDir}/out.txt', 'a', encoding = 'utf-8') as f:
        appendStr = f'{dollarAmount}, {courtNumber}\n'
        eq = appendStr == lines[i-1]
        f.write(f'{appendStr[:len(appendStr)-1]}, {eq}\n')


    print(f'{dollarAmount}, {courtName}, {courtNumber}')



LIST INCORRECT ANSWERS, AND HOW MANY CORRECT ANSWERS

In [8]:
with open(f'{mainDir}/test/ocr/out/out.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
    numPos = 0
    numLines = len(lines)
    for i in range(len(lines)):
        splitString = lines[i].split(', ')
        n = len(splitString)
        if splitString[n-1] == 'True\n':
            numPos += 1
        else:
            print(f'INCORRECT: {i} {lines[i]}' )
    print(f'{numPos}/{numLines} ({numPos/numLines}) correct')

INCORRECT: 19 $250,000.00, 4, False

INCORRECT: 33 No dollar amount found., 4, False

INCORRECT: 36 No dollar amount found., 4, False

INCORRECT: 56 $4,396.00, 1243129, False

INCORRECT: 61 $610.51, -1, False

INCORRECT: 67 $3,668.29, -1, False

INCORRECT: 76 $2,613.54, -1, False

INCORRECT: 87 $250,000.00, 2, False

81/89 (0.9101123595505618) correct


In [28]:
for i in [18,19,21,22,24,33,36,86,87]:
    pdfStr =f'{pdfDir}/{i}.pdf'
    res = ocr.ocr.process_pdf_and_find_damages(pdfStr,False)
    
    dollarAmount = getDollarAmount(res)
    splitRes = res.split(',')
    courtName = splitRes[len(splitRes)-1]
    courtNameSplit = courtName.split(' ')
    courtNumber = courtNameSplit[len(courtNameSplit)-2]
    with open(f'{outDir}/out.txt', 'a', encoding = 'utf-8') as f:
        appendStr = f'{dollarAmount}, {courtNumber}\n'
        eq = appendStr == lines[i-1]
        f.write(f'{appendStr[:len(appendStr)-1]}, {eq}\n')


    print(f'{dollarAmount}, {courtName}, {courtNumber}')

Extracting text from PDF...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...

Searching for 'damages' and the associated dollar value...
models/gemini-2.0-flash
extracted damages
$21,686.13,  "Harris County - County Civil Court at Law No. 1 ", 1
Extracting text from PDF...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...

Searching for 'damages' and the associated dollar value...
models/gemini-2.0-flash
extracted damages
$250,  "Harris County - County Civil Court at Law No. 2 ", 2
Extracting text from PDF...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...

Searching for 'damages' and the associated dollar value...
models/gemini-2.0-flash
extracted damages
$25,332.22,  "Harris County - County Civil Court at Law No. 2 

KeyboardInterrupt: 